In [1]:
#실행 파일 생성을 위한 임포트
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QMessageBox, QLineEdit, QInputDialog
from PyQt5.QtGui import QIcon, QPixmap
from PyQt5.QtCore import QCoreApplication
from PIL import ImageGrab, Image

#자동화 실행을 위한 임포트
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from PIL import Image

#이미지 비교를 위한 임포트
import numpy as np
import cv2
from matplotlib import pyplot as plt

#엑셀 저장을 위한 임포트
import pandas as pd
from bs4 import BeautifulSoup

#현재 날짜를 가져오기 위한 임포트
import datetime

#엑셀 파일 생성(D드라이브)
dot_test_text = []
dot_result_text = []
dot_version_text = []
dot_date_text = []

circle_test_text = []
circle_result_text = []
circle_version_text = []
circle_date_text = []

measure_test_text = []
measure_result_text = []
measure_version_text = []
measure_date_text = []
    
#위젯 생성 
class MyApp(QWidget):

    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):
        
        version = QPushButton(self)
        version.setText('버전 입력')
        version.clicked.connect(self.showDialog)
        version.move(10, 20)
        version.resize(130, 32)
        self.le = QLineEdit(self)
        self.le.move(155, 20)
        self.le.resize(130, 32)
        self.le.setReadOnly(True)
        
        btn1 = QPushButton(self)
        btn1.setText('점 테스트')
        btn1.clicked.connect(self.btn1_clicked)
        btn2 = QPushButton(self)
        btn2.setText('원 테스트')
        btn2.clicked.connect(self.btn2_clicked)
        btn3 = QPushButton(self)
        btn3.setText('측정/이동 테스트')
        btn3.clicked.connect(self.btn3_clicked)
        
        version_text = self.le.text()
        
        vbox = QVBoxLayout()
        vbox.addStretch(0.1)
        vbox.addWidget(btn1)
        vbox.addWidget(btn2)
        vbox.addWidget(btn3)

        self.setLayout(vbox)
        
        self.setWindowTitle('Algeomath Macro')
        self.setWindowIcon(QIcon('web.png'))
        self.setGeometry(300, 300, 300, 160)
        self.show()
    
    #버전 입력 다이얼로그
    def showDialog(self):

        text, ok = QInputDialog.getText(self, 'Input Version', '알지오매스 버전을 입력하세요:')

        if ok:
            self.le.setText(str(text))
            version_text = str(text)
            
    #이미지 비교
    def compare_img(self, ori, comp):
        testPass = True
        original = cv2.imread(ori)
        image_to_compare = cv2.imread(comp)
        if(original.shape == image_to_compare.shape):
            difference = cv2.subtract(original, image_to_compare)
            b, g, r = cv2.split(difference)
            if cv2.countNonZero(b) != 0 or cv2.countNonZero(g) != 0 or cv2.countNonZero(r) != 0:
                testPass = False
            else:
                testPass = True
        return testPass
    
    #엑셀 저장
    def dot_saveExcel(self, test, result, version):
        now = datetime.datetime.now()
        nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
        dot_test_text.append(test)
        if result == True:
            dot_result_text.append("Pass")
        else:
            dot_result_text.append("Fail")
        dot_version_text.append(version)
        dot_date_text.append(nowDatetime)
    def circle_saveExcel(self, test, result, version):
        now = datetime.datetime.now()
        nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
        circle_test_text.append(test)
        if result == True:
            circle_result_text.append("Pass")
        else:
            circle_result_text.append("Fail")
        circle_version_text.append(version)
        circle_date_text.append(nowDatetime)
    def measure_saveExcel(self, test, result, version):
        now = datetime.datetime.now()
        nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
        measure_test_text.append(test)
        if result == True:
            measure_result_text.append("Pass")
        else:
            measure_result_text.append("Fail")
        measure_version_text.append(version)
        measure_date_text.append(nowDatetime)
        
    #캡쳐
    def capture(self, save_path, x_idx, y_idx, width_idx, height_idx, i, driver, location):

        driver.save_screenshot(save_path +str(i)+".png")
        x = location['x'] + x_idx
        y = location['y'] + y_idx
        width = location['x'] + width_idx
        height = location['y'] + height_idx
        im = Image.open(save_path +str(i)+".png")
        im = im.crop((int(x), int(y), int(width), int(height)))
        im.save(save_path +str(i)+".png")
        
    #점 도구 자동화
    def btn1_clicked(self):
        options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
        options.add_argument('headless') # headless 모드 설정
        options.add_argument("window-size=1920x1080")
        testPass = True
        driver = webdriver.PhantomJS('/Users/Que/Downloads/phantomjs-2.1.1-windows/bin/phantomjs')
        driver = webdriver.Chrome('/Users/Que/Downloads/chromedriver_win32/chromedriver.exe', options=options)

        # url에 접근
        driver.get('https://www.algeomath.kr/algeomath/app/make.do')
        driver.maximize_window()
        driver.implicitly_wait(3)

        #그리드 없애기
        driver.find_element_by_id('gridSetBtn').click()
        driver.find_element_by_xpath('//*[@id="gridTab"]/div[2]/div[1]/label[2]/span').click()

        #점 메뉴 클릭
        driver.find_element_by_class_name('pointBtn').click()

        driver.implicitly_wait(3)

        #점이 찍히는 위치: 점메뉴에서 오른쪽으로 1000, 아래로 200
        action = ActionChains(driver)
        element = driver.find_element_by_id('geoGraph')
        action.move_to_element_with_offset(element, 1000, 200).double_click().perform()
        driver.implicitly_wait(3)
        driver.find_element_by_class_name('basicBtn').click()
        driver.implicitly_wait(3)
        action.move_to_element_with_offset(element, 1000, 200).double_click().perform()
        driver.implicitly_wait(3)

        #########################색상 클릭
        driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[1]').click()
        for i in range(1, 11):
            driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[2]/div['+str(i)+']').click()
            location = driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[7]').location
            
            self.capture("C:/Users/Que/자동화도구 개발/dot/color/color", 0, -60, 50, -20, i, driver, location)
            
            testPass = self.compare_img("dot/color/color"+str(i)+".png", "dot/original-color/color"+str(i)+".png")
                    
        self.dot_saveExcel("dot color test", testPass, self.le.text())
        
        driver.implicitly_wait(3)
        
        testPass = True
        #########################모양 클릭
        driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[3]').click()
        for i in range(1, 16):
            driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[4]/div[1]/ul[2]/li[1]/div['+str(i)+']').click()
            location = driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[7]').location
    
            self.capture("C:/Users/Que/자동화도구 개발/dot/shape/shape", 0, -60, 50, -20, i, driver, location)
        
            testPass = self.compare_img("dot/shape/shape"+str(i)+".png", "dot/original-shape/shape"+str(i)+".png")
                    
        self.dot_saveExcel("dot shape test", testPass, self.le.text())
        
        testPass = True
        #########################크기 변경
        size = driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[4]/div[2]/input').click()
        action.drag_and_drop_by_offset(size, 30, 0)

        #########################이름 변경
        driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[5]').click()
        driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[6]/input').send_keys('lgeoMath')
        location = driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[1]').location
        
        self.capture("C:/Users/Que/자동화도구 개발/dot/name/change_name", 0, -140, 240, 100, 1, driver, location)
        
        testPass = self.compare_img("dot/name/change_name1.png", "dot/original-name/change_name1.png")
        
        self.dot_saveExcel("dot size and name change test", testPass, self.le.text())
            
        testPass = True
        #########################숨기기
        driver.find_element_by_xpath('//*[@id="algeoPropPoint"]/div[2]/div/div[11]').click()
        location = driver.find_element_by_class_name('pointBtn').location
        
        self.capture("C:/Users/Que/자동화도구 개발/dot/hide/hide", 870, 70, 870+240, 60+240, 1, driver, location)
        
        testPass = self.compare_img("dot/hide/hide1.png", "dot/original-hide/hide1.png")
                
        self.dot_saveExcel("dot hide test", testPass, self.le.text())
        
        testPass = True
        #########################교점/대상 위의 점/ 중점
        #교점/대상 위의 점/ 중점을 만들기 위한 선분 그리기
        driver.find_element_by_class_name('lineBtn').click()

        element = driver.find_element_by_id('geoGraph')
        #교점 선분
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 200).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 400).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 400).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 200).click().perform()

        #대상위의 점 선분
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 500).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 500).click().perform()

        #중점 선분
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 600).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 600).click().perform()

        #교점 만들기
        driver.find_element_by_class_name('pointBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-point-group"]/div[2]/ul/li[2]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 230, 230).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 470, 225).click().perform()

        #대상 위의 점 만들기
        driver.find_element_by_class_name('pointBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-point-group"]/div[2]/ul/li[3]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 500).click().perform()

        #중점 만들기
        driver.find_element_by_class_name('pointBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-point-group"]/div[2]/ul/li[4]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 600).click().perform()

        #########################이미지
        driver.find_element_by_class_name('pointBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-point-group"]/div[2]/ul/li[5]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 1000, 200).click().perform()
        driver.find_element_by_xpath('//*[@id="dlgCreateImage"]/div[2]/div[1]/div[2]/label[2]').click()
        driver.find_element_by_class_name('txtImageUrl').send_keys('http://blogfiles.naver.net/MjAxODExMTNfNiAg/MDAxNTQyMDc0MTI5MTY5.xX1zAMkIMaF9RsCehI2f4vRxWG_v0f8KRQPIr-7i_Dgg.ZLCV5YteAMvnrA9_WSXhhSk-57q1E2fmeuUqA7ote8Eg.PNG.ideabigbang/O7M1OGV2.png')
        driver.find_element_by_xpath('//*[@id="dlgCreateImage"]/div[2]/div[3]/input[1]').click()
        
        time.sleep(1)
        #########################캡쳐
        location = driver.find_element_by_id('geoGraph').location
        driver.save_screenshot("C:/Users/Que/자동화도구 개발/dot/all/all.png")
        
        testPass = self.compare_img("dot/all/all.png", "dot/original-all/all.png")
        
        self.dot_saveExcel("dot make and img test", testPass, self.le.text())
        
        testPass = True
        #########################동영상
        driver.find_element_by_class_name('pointBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-point-group"]/div[2]/ul/li[6]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 1000, 500).click().perform()
        driver.find_element_by_class_name('txtVideoUrl').send_keys('https://www.youtube.com/watch?v=cbSHi_gI1j0&t=7s')
        driver.find_element_by_xpath('//*[@id="dlgCreateVideo"]/div[2]/div[2]/input[1]').click()
        #선택 메뉴로 확인
        driver.find_element_by_class_name('basicBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 810, 530).click().perform()
        time.sleep(1)
        #캡쳐
        location = driver.find_element_by_id('geoGraph').location
    
        self.capture("C:/Users/Que/자동화도구 개발/dot/all/video", 700, 550, 800, 800, 1, driver, location)

        testPass = self.compare_img("dot/all/video1.png", "dot/original-all/video1.png")
                
        self.dot_saveExcel("dot video test", testPass, self.le.text())
            
        #########################엑셀 파일 저장
        result = {'test':dot_test_text, 'result':dot_result_text, 'version':dot_version_text, 'date':dot_date_text}
        df = pd.DataFrame(result)
        RESULT_PATH = 'D:/'
        outputFileName = 'dot-test.xlsx'
        df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')

        #########################이미지 비교 이상 유무 확인
        reply = QMessageBox.question(self, 'Message', 'Test종료 (D드라이브 dot-test.xlsx로 결과를 확인하세요.)', QMessageBox.Yes)
        
        
    #원 도구 자동화
    def btn2_clicked(self):
        options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
        options.add_argument('headless') # headless 모드 설정
        options.add_argument("window-size=1920x1080")
        testPass = True
        driver = webdriver.PhantomJS('/Users/Que/Downloads/phantomjs-2.1.1-windows/bin/phantomjs')
        driver = webdriver.Chrome('/Users/Que/Downloads/chromedriver_win32/chromedriver.exe', options=options)
        
        # url에 접근
        driver.get('https://www.algeomath.kr/algeomath/app/make.do')
        driver.maximize_window()
        driver.implicitly_wait(3)

        #그리드 없애기
        driver.find_element_by_id('gridSetBtn').click()
        driver.find_element_by_xpath('//*[@id="gridTab"]/div[2]/div[1]/label[2]/span').click()

        #원 메뉴 클릭
        driver.find_element_by_class_name('circleBtn').click()

        driver.implicitly_wait(3)
        
        location = driver.find_element_by_id('geoGraph').location

        #########################원 그리기
        element = driver.find_element_by_id('geoGraph')
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 150).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 100).click().perform()
        driver.implicitly_wait(3)

        #########################컨버스 그리기
        driver.find_element_by_class_name('circleBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-circle-group"]/div[2]/ul/li[2]/div').click()

        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 150).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 100).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 150).click().perform()
        driver.implicitly_wait(3)

        #########################세 점으로 원 그리기
        driver.find_element_by_class_name('circleBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-circle-group"]/div[2]/ul/li[3]/div').click()

        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 600, 150).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 150).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 550, 200).click().perform()

        #########################중심과 반지름
        driver.find_element_by_class_name('circleBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-circle-group"]/div[2]/ul/li[4]/div').click()

        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 800, 150).click().perform()
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="radius"]').send_keys(Keys.BACKSPACE)
        driver.find_element_by_xpath('//*[@id="radius"]').send_keys('2')
        driver.find_element_by_xpath('//*[@id="algeoModalGenericCircle"]/div[2]/div[2]/input[1]').click()

        #########################호
        driver.find_element_by_class_name('circleBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-circle-group"]/div[2]/ul/li[5]/div').click()

        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 500).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 50, 400).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 400).click().perform()

        #########################부채꼴
        driver.find_element_by_class_name('circleBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-circle-group"]/div[2]/ul/li[6]/div').click()

        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 400, 500).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 350, 400).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 400).click().perform()

        #########################활꼴
        driver.find_element_by_class_name('circleBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-circle-group"]/div[2]/ul/li[7]/div').click()

        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 700, 500).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 650, 400).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 800, 400).click().perform()

        #########################캡쳐
        location = driver.find_element_by_id('geoGraph').location
        driver.save_screenshot("C:/Users/Que/자동화도구 개발/circle/all/all.png")

        #기본 선택메뉴 클릭
        driver.find_element_by_class_name('basicBtn').click()

        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 140, 120).click().perform()
        
        testPass = self.compare_img("circle/all/all.png", "circle/original-all/all.png")
        self.circle_saveExcel("circle draw test", testPass, self.le.text())
        
        
        testPass = True
        #########################색상 변경
        driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[1]').click()
        for i in range(1, 11):
            driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[2]/div['+str(i)+']').click()
            
            self.capture("C:/Users/Que/자동화도구 개발/circle/color/color", 0, 100, 200, 130, i, driver, location)
            testPass = self.compare_img("circle/color/color"+str(i)+".png", "circle/original-color/color"+str(i)+".png")

        self.circle_saveExcel("circle color test", testPass, self.le.text())
        #다음 진행을 위해 빨간선으로 변경
        driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[2]/div[1]').click()
        driver.implicitly_wait(3)
        
        testPass = True
        #########################선 모양 변경
        #선 스타일 설정
        driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[3]').click()
        for i in range(1, 7):
            driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[4]/div[1]/div[2]').click()
            driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[4]/div[1]/div[3]/div['+ str(i) + ']').click()
            
            self.capture("C:/Users/Que/자동화도구 개발/circle/line-style/line-style", 0, 100, 200, 130, i, driver, location)
            testPass = self.compare_img("circle/line-style/line-style"+str(i)+".png", "circle/original-line-style/line-style"+str(i)+".png")
        self.circle_saveExcel("circle line style test", testPass, self.le.text())
        
        #선 굵기
        line_thick = driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[4]/div[2]/div[1]/input')
        action = ActionChains(driver)
        action.move_to_element_with_offset(line_thick, 50, 0).click().perform()

        #투명도
        opacity = driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[4]/div[2]/div[2]/input')
        action = ActionChains(driver)
        action.move_to_element_with_offset(opacity, 50, 0).click().perform()
    
        testPass = True
        #########################선 패턴
        #패턴 스타일
        driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[6]').click()
        for i in range(1, 9):
            driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[7]/div[1]/div[2]/div['+ str(i) + ']').click()
        
            self.capture("C:/Users/Que/자동화도구 개발/circle/pattern/pattern", 0, 100, 200, 130, i, driver, location)
            testPass = self.compare_img("circle/pattern/pattern"+str(i)+".png", "circle/original-pattern/pattern"+str(i)+".png")
        self.circle_saveExcel("circle pattern test", testPass, self.le.text())
        
        testPass = True
        #그라데이션
        driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[7]/div[3]/div[2]/label[1]').click()
        for i in range(1, 5):
            driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[7]/div[4]/div/div[1]/div['+ str(i) +']').click()
            
            self.capture("C:/Users/Que/자동화도구 개발/circle/gradation/gradation", 0, 100, 200, 130, i, driver, location)
            testPass = self.compare_img("circle/gradation/gradation"+str(i)+".png", "circle/original-gradation/gradation"+str(i)+".png")

        driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[7]/div[3]/div[2]/label[2]').click()
        for i in range(1, 6):
            driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[7]/div[4]/div/div[2]/div['+ str(i) +']').click()
            
            self.capture("C:/Users/Que/자동화도구 개발/circle/gradation/gradation", 0, 100, 200, 130, i+4, driver, location)
            testPass = self.compare_img("circle/gradation/gradation"+str(i+4)+".png", "circle/original-gradation/gradation"+str(i+4)+".png")
            
        self.circle_saveExcel("circle gradation test", testPass, self.le.text())
        
        #########################숨기기
        driver.find_element_by_xpath('//*[@id="algeoPropLine"]/div[2]/div/div[9]').click()
        
        self.capture("C:/Users/Que/자동화도구 개발/circle/hide/hide", 0, 100, 200, 130, 1, driver, location)
        testPass = self.compare_img("circle/hide/hide1.png", "circle/original-hide/hide1.png")
        self.circle_saveExcel("circle hide test", testPass, self.le.text())
        
        #########################엑셀 파일 저장
        result = {'test':circle_test_text, 'result':circle_result_text, 'version':circle_version_text, 'date':circle_date_text}
        df = pd.DataFrame(result)
        RESULT_PATH = 'D:/'
        outputFileName = 'circle-test.xlsx'
        df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')
        
        #########################이미지 비교 이상 유무 확인
        reply = QMessageBox.question(self, 'Message', 'Test종료 (D드라이브 circle-test.xlsx로 결과를 확인하세요.)', QMessageBox.Yes)
    
    
    #측정/이동 도구 자동화
    def btn3_clicked(self):
        options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
        options.add_argument('headless') # headless 모드 설정
        options.add_argument("window-size=1920x1080")
        testPass = True
        driver = webdriver.PhantomJS('/Users/Que/Downloads/phantomjs-2.1.1-windows/bin/phantomjs')
        driver = webdriver.Chrome('/Users/Que/Downloads/chromedriver_win32/chromedriver.exe', options=options)
        
        # url에 접근
        driver.get('https://www.algeomath.kr/algeomath/app/make.do')
        driver.maximize_window()
        driver.implicitly_wait(3)

        #그리드 없애기
        driver.find_element_by_id('gridSetBtn').click()
        driver.find_element_by_xpath('//*[@id="gridTab"]/div[2]/div[1]/label[2]/span').click()

        location = driver.find_element_by_id('geoGraph').location
        element = driver.find_element_by_id('geoGraph')
        
        location = driver.find_element_by_id('geoGraph').location

        #########################길이 측정
        driver.find_element_by_class_name('lineBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 100).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 100).click().perform()

        driver.find_element_by_class_name('lengthBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 100).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 100).click().perform()

        self.capture("C:/Users/Que/자동화도구 개발/measure/length/length", 50, 50, 350, 120, 1, driver, location)
        testPass = self.compare_img("measure/length/length1.png", "measure/original-length/length1.png")
        self.measure_saveExcel("measure length test", testPass, self.le.text())

        testPass = True
        #########################각도 측정
        driver.find_element_by_class_name('lineBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 200).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 300).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 300).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 300).click().perform()

        driver.find_element_by_class_name('lengthBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-length-group"]/div[2]/ul/li[2]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 200).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 300).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 300).click().perform()
        
        self.capture("C:/Users/Que/자동화도구 개발/measure/angle/angle", 50, 170, 350, 350, 1, driver, location)
        testPass = self.compare_img("measure/angle/angle1.png", "measure/original-angle/angle1.png")
        self.measure_saveExcel("measure angle test", testPass, self.le.text())

        testPass = True
        #########################넓이 측정
        driver.find_element_by_class_name('polygonBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 350).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 350).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 300, 400).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 400).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 350).click().perform()

        driver.find_element_by_class_name('lengthBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-length-group"]/div[2]/ul/li[3]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 375).click().perform()

        self.capture("C:/Users/Que/자동화도구 개발/measure/area/area", 50, 350, 350, 450, 1, driver, location)
        testPass = self.compare_img("measure/area/area1.png", "measure/original-area/area1.png")
        self.measure_saveExcel("measure area test", testPass, self.le.text())


        testPass = True
        #########################점 대칭 
        driver.find_element_by_class_name('lineBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 450).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 500).click().perform()
        driver.find_element_by_class_name('pointBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 500).click().perform()


        driver.find_element_by_class_name('lengthBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-length-group"]/div[2]/ul/li[4]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 150, 475).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 500).click().perform()

        #########################선 대칭 
        driver.find_element_by_class_name('pointBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 650).click().perform()
        driver.find_element_by_class_name('lineBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 200, 650).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 700).click().perform()

        driver.find_element_by_class_name('lengthBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-length-group"]/div[2]/ul/li[5]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 100, 650).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 150, 675).click().perform()
        
        self.capture("C:/Users/Que/자동화도구 개발/measure/symmetry/symmetry", 50, 420, 350, 750, 1, driver, location)
        testPass = self.compare_img("measure/symmetry/symmetry1.png", "measure/original-symmetry/symmetry1.png")
        self.measure_saveExcel("measure symmetry test", testPass, self.le.text())

        testPass = True
        #########################회전
        driver.find_element_by_class_name('polygonBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 100).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 450, 150).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 600, 170).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 100).click().perform()
        driver.find_element_by_class_name('pointBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 450, 200).click().perform()

        driver.find_element_by_class_name('lengthBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-length-group"]/div[2]/ul/li[6]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 475, 140).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 450, 200).click().perform()

        driver.find_element_by_xpath('//*[@id="algeoModalRotation"]/div[2]/div[3]/input[1]').click()
        
        self.capture("C:/Users/Que/자동화도구 개발/measure/rotate/rotate", 400, 50, 650, 300, 1, driver, location)
        testPass = self.compare_img("measure/rotate/rotate1.png", "measure/original-rotate/rotate1.png")
        self.measure_saveExcel("measure rotate test", testPass, self.le.text())

        testPass = True
        #########################평행이동
        driver.find_element_by_class_name('lineBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-line-group"]/div[2]/ul/li[10]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 450).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 600, 350).click().perform()
        driver.find_element_by_class_name('pointBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 500).click().perform()

        driver.find_element_by_class_name('lengthBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-length-group"]/div[2]/ul/li[7]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 500).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 550, 400).click().perform()
        
        self.capture("C:/Users/Que/자동화도구 개발/measure/parallel shift/parallel shift", 400, 300, 650, 470, 1, driver, location)
        testPass = self.compare_img("measure/parallel shift/parallel shift1.png", "measure/original-parallel shift/parallel shift1.png")
        self.measure_saveExcel("measure parallel shift test", testPass, self.le.text())
        
        testPass = True
        #########################점을 중심으로 확대
        driver.find_element_by_class_name('polygonBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 550, 550).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 600, 650).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 500, 650).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 550, 550).click().perform()
        driver.find_element_by_class_name('pointBtn').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 450, 675).click().perform()

        driver.find_element_by_class_name('lengthBtn').click()
        driver.find_element_by_xpath('//*[@id="geometry-length-group"]/div[2]/ul/li[8]/div').click()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 550, 600).click().perform()
        action = ActionChains(driver)
        action.move_to_element_with_offset(element, 450, 675).click().perform()
        driver.find_element_by_xpath('//*[@id="rate"]').send_keys(Keys.BACKSPACE)
        driver.find_element_by_xpath('//*[@id="rate"]').send_keys('5')
        driver.find_element_by_xpath('//*[@id="algeoModalExpandToPoint"]/div[2]/div[2]/input[1]').click()
        
        self.capture("C:/Users/Que/자동화도구 개발/measure/expansion/expansion", 650, 50, 800, 800, 1, driver, location)
        testPass = self.compare_img("measure/expansion/expansion1.png", "measure/original-expansion/expansion1.png")
        self.measure_saveExcel("measure expansion test", testPass, self.le.text())
        
        time.sleep(1)
        
        #########################엑셀 파일 저장
        result = {'test':measure_test_text, 'result':measure_result_text, 'version':measure_version_text, 'date':measure_date_text}
        df = pd.DataFrame(result)
        RESULT_PATH = 'D:/'
        outputFileName = 'measure-test.xlsx'
        df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')
        
        #########################이미지 비교 이상 유무 확인
        reply = QMessageBox.question(self, 'Message', 'Test종료 (D드라이브 measure-test.xlsx로 결과를 확인하세요.)', QMessageBox.Yes)
        
#메인 함수
if __name__ == '__main__':

    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())

C:\Users\Que\Anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


SystemExit: 0

C:\Users\Que\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
